In [1]:
from tensorflow.keras.applications import InceptionV3
from tensorflow import keras
from tensorflow.keras.layers import *
from prepare_data import *
from datetime import datetime
from tensorflow.keras.applications.inception_v3 import preprocess_input
import albumentations as A
import tensorflow as tf

In [2]:
dataset = Dataset.carabid
current_time = datetime.now().strftime("%d%m%Y-%H%M%S")
raw_train, raw_val = prep_data_single(dataset, 8)

Found 63364 files belonging to 291 classes.
Using 53860 files for training.
Found 63364 files belonging to 291 classes.
Using 9504 files for validation.


In [3]:
def apply_aug(images):
    aug_imgs = []
    for img in images:
        aug_imgs.append(augment(image=img)["image"])
    return np.array(aug_imgs)

def process_data(images, labels):
    aug_imgs = tf.numpy_function(apply_aug, [images], tf.float32)
    return aug_imgs, labels

extractor_train = raw_train.map(process_data)
extractor_val = raw_val.map(process_data)

In [3]:
gray_aug = A.Compose([A.ToGray(p=1.0)])
blur_aug = A.Compose([A.Blur(p=1.0)])

def apply_aug_gray(images):
    aug_imgs = []
    for img in images:
        aug_imgs.append(gray_aug(image=img)["image"])
    return np.array(aug_imgs)

def apply_aug_blur(images):
    aug_imgs = []
    for img in images:
        aug_imgs.append(blur_aug(image=img)["image"])
    return np.array(aug_imgs)

def process_data_gray(images, labels):
    aug_imgs = tf.numpy_function(apply_aug_gray, [images], tf.float32)
    return aug_imgs, labels

def process_data_blur(images, labels):
    aug_imgs = tf.numpy_function(apply_aug_blur, [images], tf.float32)
    return aug_imgs, labels

standard_train = raw_train
gray_train = raw_train.map(process_data_gray)
blur_train = raw_train.map(process_data_blur)
standard_val = raw_val
gray_val = raw_val.map(process_data_gray)
blur_val = raw_val.map(process_data_blur)

train_datasets = [standard_train, gray_train, blur_train]
val_datasets = [standard_val, gray_val, blur_val]

extractor_train = tf.data.Dataset.sample_from_datasets(train_datasets)
extractor_val = tf.data.Dataset.sample_from_datasets(val_datasets)

In [4]:
inception = InceptionV3(classifier_activation=None)
inception.trainable = True

inputs = keras.Input(shape=(299, 299, 3))
flip_aug = experimental.preprocessing.RandomFlip()(inputs)
rotate_aug = experimental.preprocessing.RandomRotation(0.5)(flip_aug)
preprocessing = preprocess_input(rotate_aug)
extractor = inception(preprocessing, training=False)
inception_model = keras.Model(inputs=inputs, outputs=extractor)

In [5]:
extractor_logdir = "logs/combo/{0}_{1}/extractor".format(str(dataset), current_time)
extractor_tensorboard_callback = keras.callbacks.TensorBoard(log_dir=extractor_logdir)

extractor_model_path = "models/combo/{0}_{1}/extractor/savefile.hdf5".format(str(dataset), current_time)
extractor_model_save_callback = keras.callbacks.ModelCheckpoint(filepath=extractor_model_path, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

In [6]:
feature_extractor = keras.Sequential([inception_model, keras.layers.Dense(num_classes(dataset), activation='softmax')])
feature_extractor.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
feature_extractor.fit(extractor_train, validation_data=extractor_val, callbacks=[extractor_tensorboard_callback, extractor_model_save_callback], epochs=15)

Epoch 1/15
  20199/Unknown - 2507s 124ms/step - loss: 1.5637 - accuracy: 0.5919
Epoch 1: val_accuracy improved from -inf to 0.68459, saving model to models/combo/Dataset.carabid_13042022-171033/extractor\savefile.hdf5
20199/20199 [==============================] - 2644s 130ms/step - loss: 1.5637 - accuracy: 0.5919 - val_loss: 1.0660 - val_accuracy: 0.6846
Epoch 2/15
20199/20199 [==============================] - ETA: 0s - loss: 0.5939 - accuracy: 0.8134
Epoch 2: val_accuracy improved from 0.68459 to 0.78363, saving model to models/combo/Dataset.carabid_13042022-171033/extractor\savefile.hdf5
20199/20199 [==============================] - 2460s 122ms/step - loss: 0.5939 - accuracy: 0.8134 - val_loss: 0.7411 - val_accuracy: 0.7836
Epoch 3/15
20199/20199 [==============================] - ETA: 0s - loss: 0.3883 - accuracy: 0.8759
Epoch 3: val_accuracy improved from 0.78363 to 0.78855, saving model to models/combo/Dataset.carabid_13042022-171033/extractor\savefile.hdf5
20199/20199 [=======

KeyboardInterrupt: 

In [3]:
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.RandomGamma(p=0.5),
    A.ToGray(p=0.3),
    A.Blur(p=0.3)
])

train_gen, val_gen = prep_data_augmented(dataset, extractor_train, extractor_val, 8, 6, augment=augment)
del extractor_train
del extractor_val

In [4]:
extractor_model_path = "models/carabid/CARABID-EXTRACTOR/extractor/savefile.hdf5"

feature_extractor = keras.models.load_model(extractor_model_path)
inception_model = feature_extractor.layers[0].layers[-1]
inception_model.trainable = False

classifier_model = keras.Sequential([
    InputLayer(input_shape=(None, 299, 299, 3)),
    TimeDistributed(inception_model),
    Bidirectional(LSTM(1000)),
    Dropout(0.5),
    Dense(1000, activation='relu'),
    Dense(train_gen.num_classes(), activation='softmax')
])

classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(classifier_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, None, 1000)       23851784  
 ibuted)                                                         
                                                                 
 bidirectional (Bidirectiona  (None, 2000)             16008000  
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 2000)              0         
                                                                 
 dense (Dense)               (None, 1000)              2001000   
                                                                 
 dense_1 (Dense)             (None, 291)               291291    
                                                                 
Total params: 42,152,075
Trainable params: 18,300,291
No

In [5]:
logdir = "logs/unfiltered/{0}_{1}/classifier".format(str(dataset), current_time)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model_path = "models/unfiltered/{0}_{1}/classifier/savefile.hdf5".format(str(dataset), current_time)
model_save_callback = keras.callbacks.ModelCheckpoint(filepath=model_path, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

In [6]:
class ShuffleCallback(keras.callbacks.Callback):
    def __init__(self, generator):
        self._generator = generator
        
    def on_epoch_end(self, epoch, logs=None):
        self._generator.shuffle()
    
train_shuffle_callback = ShuffleCallback(train_gen)
val_shuffle_callback = ShuffleCallback(val_gen)

In [7]:
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
classifier_model.fit(train_gen, validation_data=val_gen, callbacks=[tensorboard_callback, model_save_callback, train_shuffle_callback, val_shuffle_callback], epochs=20)

Epoch 1/20
6733/6733 [==============================] - ETA: 0s - loss: 1.9422 - accuracy: 0.6400
Epoch 1: val_accuracy improved from -inf to 0.86385, saving model to models/unfiltered/Dataset.carabid_03042022-220209/classifier\savefile.hdf5
6733/6733 [==============================] - 963s 141ms/step - loss: 1.9422 - accuracy: 0.6400 - val_loss: 0.5409 - val_accuracy: 0.8638
Epoch 2/20
6733/6733 [==============================] - ETA: 0s - loss: 0.4106 - accuracy: 0.8950
Epoch 2: val_accuracy improved from 0.86385 to 0.89689, saving model to models/unfiltered/Dataset.carabid_03042022-220209/classifier\savefile.hdf5
6733/6733 [==============================] - 953s 141ms/step - loss: 0.4106 - accuracy: 0.8950 - val_loss: 0.3701 - val_accuracy: 0.8969
Epoch 3/20
6733/6733 [==============================] - ETA: 0s - loss: 0.2708 - accuracy: 0.9257
Epoch 3: val_accuracy improved from 0.89689 to 0.90583, saving model to models/unfiltered/Dataset.carabid_03042022-220209/classifier\savefile

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import tensorflow as tf

class Classifier:
    def __init__(self, model_path, category_dict):
        self.model = keras.models.load_model(model_path)
        self.category_dict = category_dict
        self.augment = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Rotate(p=0.5),
            A.RandomBrightnessContrast(p=1.0),
            A.RandomGamma(p=0.5),
            A.ToGray(p=0.3),
            A.Blur(p=0.3),
        ])

    def classify(self, img_path, num_augments=6):
        input_array = self._make_augment_array(np.asarray(img_to_array(load_img(img_path, target_size=(299, 299))), dtype='uint8'), num_augments)
        prediction = self.model(input_array)
        print(tf.math.argmax(prediction, axis=1)[0].numpy())

    def _make_augment_array(self, img, num_augments):
        result = [preprocess_input(img)]
        for i in range(num_augments - 1):
            result.append(preprocess_input(self.augment(image=img)["image"]))
        return np.asarray([result])

In [ ]:
model = Classifier("models/carabid/CARABID_AUG-6_CLASSIFIER/classifier/savefile.hdf5", {})

img_path = "datasets/carabid/9581584/d135s0001.jpg"

model.classify(img_path)